In [1]:
import pandas as pd
import numpy as np

from scipy.stats import norm
from scipy.optimize import brentq
from scipy import interpolate
from scipy.integrate import quad

import datetime as dt

import matplotlib.pylab as plt

## SPX

In [2]:
df = pd.read_csv(r'SPX_options.csv')
df['mid'] = 0.5*(df['best_bid'] + df['best_offer'])
df['strike'] = df['strike_price'] * 0.001

df['payoff'] = df['cp_flag'].map(lambda x: 'call' if x == 'C' else 'put')

spx = df[(df.exdate == 20210115)]

In [3]:
spx.tail()

,date,exdate,cp_flag,strike_price,best_bid,best_offer,exercise_style,mid,strike,payoff
1555,20201201,20210115,P,5000000,1330.8,1346.6,E,1338.70,5000.0,put
1556,20201201,20210115,P,5100000,1428.3,1451.2,E,1439.75,5100.0,put
1557,20201201,20210115,P,5200000,1528.3,1551.2,E,1539.75,5200.0,put
1558,20201201,20210115,P,5300000,1628.2,1651.1,E,1639.65,5300.0,put
1559,20201201,20210115,P,5400000,1728.1,1751.0,E,1739.55,5400.0,put


## T for part 3

In [4]:
today = dt.date(2020, 12, 1)
expiry = dt.date(2021, 1, 15)

T = (expiry-today).days/365.0

## Risk-free rate

In [5]:
rf = pd.read_csv(r'zero_rates_20201201.csv')

In [6]:
x = rf['days']
y = rf['rate']
f = interpolate.interp1d(x,y)
r = f(T*365)/100

In [7]:
r

0.0020510755555555554

## Other data

In [8]:
K = 3660
S = 3662.45
F = S*np.exp(r*T)

In [9]:
F

3663.3762493669747

# Black-Scholes

In [10]:
def BlackScholesCall(S, K, r, sigma, T):
    d1 = (np.log(S/K)+(r+sigma**2/2)*T) / (sigma*np.sqrt(T))
    d2 = d1 - sigma*np.sqrt(T)
    return S*norm.cdf(d1) - K*np.exp(-r*T)*norm.cdf(d2)

def BlackScholesPut(S, K, r, sigma, T):
    return BlackScholesCall(S,K,r,sigma,T)- S + K*np.exp(-r*T)

In [11]:
# Implied Vol
def impliedCallVolatility(S, K, r, price, T):
    try:
        impliedVol = brentq(lambda x: price -
                        BlackScholesCall(S, K, r, x, T),
                        1e-6, 1)
    except Exception:
        impliedVol = np.nan
 
    return impliedVol

def impliedPutVolatility(S, K, r, price, T):
    try:
        impliedVol = brentq(lambda x: price -
                        BlackScholesPut(S, K, r, x, T),
                        1e-6, 1)
    except Exception:
        impliedVol = np.nan

    return impliedVol

In [12]:
## At The Money Vol
atm_call = spx[(spx.strike == K)]
atm_call = atm_call[(atm_call.cp_flag == "C")]

atm_put = spx[(spx.strike == K)]
atm_put = atm_put[(atm_put.cp_flag == "P")]

sigma_call = impliedCallVolatility(S, K, r , atm_call.mid, T)
sigma_put = impliedPutVolatility(S, K, r, atm_put.mid, T)

sigma = (sigma_call + sigma_put)/2

In [13]:
# BSM Integrated Variance
def callintegrand(K, S, r, T, sigma):
    price = BlackScholesCall(S, K, r, sigma, T) / K**2
    return price

def putintegrand(K, S, r, T, sigma):
    price = BlackScholesPut(S, K, r, sigma, T) / K**2
    return price

I_put = quad(lambda x: putintegrand(x, S, r, T, sigma), 0.0, F)
I_call = quad(lambda x: callintegrand(x, S, r, T, sigma), F, 5000)
E_var_BSM = 2*np.exp(r*T)*(I_put[0] + I_call[0])

print('BSM Integrated Variance: %.9f' % E_var_BSM)

BSM Integrated Variance: 0.004236501


In [14]:
# Model-free vol
sigma_BSM = np.sqrt(E_var_BSM/T)

In [37]:
sigma_BSM

0.1853718792209719

In [15]:
def Price_BSM(S, r, T, sigma):
    return ((S ** (1/3)) * np.exp((1/3) * r - (1/9) * (sigma ** 2) * T) +\
            (3/2) * np.log(S) +\
            (3/2) * (r - ((sigma ** 2) / 2)) * T + 10) * (np.exp(- r * T))

In [16]:
P1 = Price_BSM(S, r, T, sigma_BSM)

print('BSM Pricing: %.9f' % P1)

BSM Pricing: 37.714081753


# Bachelier

In [17]:
def BachelierCall(S, K, r, sigma, T):
    F = S * np.exp(r*T)
    d = (F-K) / (F*sigma*np.sqrt(T))
    disc = np.exp(-r*T)
    return disc*((F-K)*norm.cdf(d)+F*sigma*np.sqrt(T)*norm.pdf(d))

def BachelierPut(S, K, r, sigma, T):
    F = S * np.exp(r*T)
    d = (F-K) / (F*sigma*np.sqrt(T))
    disc = np.exp(-r*T)
    return disc*((K-F)*norm.cdf(-d)+F*sigma*np.sqrt(T)*norm.pdf(-d))

In [18]:
# BACH Model Integrated Variance
def callintegrandb(K, S, r, T, sigma):
    price = BachelierCall(S, K, r, sigma, T) / K**2
    return price

def putintegrandb(K, S, r, T, sigma):
    price = BachelierPut(S, K, r, sigma, T) / K**2
    return price

I_put = quad(lambda x: putintegrandb(x, S, r, T, sigma), 0.0, F)
I_call = quad(lambda x: callintegrandb(x, S, r, T, sigma), F, 5000)
E_var_BACH = 2*np.exp(r*T)*(I_put[0] + I_call[0])

print('BACH Integrated Variance: %.9f' % E_var_BACH)

BACH Integrated Variance: 0.004263812


In [19]:
sigma_BACH = np.sqrt(E_var_BACH/T)

In [20]:
sigma_BACH

0.18596842908957534

In [21]:
import scipy as sp
def raw_moment(mean, sigma, moment):
    hyp1 = sp.special.hyp1f1(-moment/2, 0.5, - (mean ** 2)/(2 * sigma ** 2))
    c_hyp1 = np.sqrt(np.pi) / sp.special.gamma((1-moment)/2)
    
    hyp2 = sp.special.hyp1f1((1-moment)/2, 1.5, - (mean ** 2)/(2 * sigma ** 2))
    c_hyp2 = (mean/sigma) * np.sqrt(2 * np.pi) / sp.special.gamma((-moment)/2)
    c_hyp2 = complex(0, c_hyp2)
    
    hyp = (c_hyp1 * hyp1) + (c_hyp2 * hyp2)

    raw_m = (complex(0,sigma)**moment) * (2 ** (moment/2)) * hyp
    return np.real(raw_m)

def expected_log_approx(mean, sigma): # not accurate
    return np.log(mean) - (sigma**2)/(raw_moment(mean,sigma,2))

def log_integral(mean, sigma):
    return sp.integrate.quad(lambda x: np.log(mean + sigma * x) * sp.stats.norm.pdf(x), -(mean/sigma), np.inf)[0]

def BA_Payoff_Func(S, r, sigma, T):
    F = S * np.exp(r*T)
    mean = F
    sigma = sigma * F * np.sqrt(T)
#     value = raw_moment(mean, sigma, 1/3) + (1.5 * expected_log_approx(mean, sigma)) + 10

## To calculate the payoff of BACH, we get expected raw 1/3 moment of a normal distribution, and the log integral
    value = raw_moment(mean, sigma, 1/3) + (1.5 * log_integral(mean, sigma)) + 10
    
    return value * np.exp(-r*T)


P2 = BA_Payoff_Func(S = 3662.45, r = 0.20510755555555554/100, sigma=sigma_BACH,  T = T)
print('BACH Pricing: %.9f' % P2)

BACH Pricing: 37.704720822


# Static-replication

In [22]:
from option_pricer import MustSet, Black76, SABR # SABR class

In [23]:
class SABR(Black76):
    """
    SABR model implemented via calculation of blackscholes implied vol, then passed that vol back to black 76 model
    """
    alpha = MustSet()
    beta = MustSet()
    rho = MustSet()
    nu = MustSet()

    def __repr__(self):
        out_str = str(self.__class__.__base__.__name__)
        out_str += ': ' + str(self.__class__.__name__) + '\n\t'
        return out_str + self._params_status_str(['F',
                                                  'K',
                                                  'r',
                                                  'alpha',
                                                  'beta',
                                                  'rho',
                                                  'nu',
                                                  'T'])

    def __init__(self, F=None, K=None, r=None, alpha=None, beta=None, rho=None,
                 nu=None, T=None, verbose=False):
        super().__init__(F=F, K=K, r=r, sigma=None, T=T, verbose=verbose)
        self.alpha = alpha
        self.beta = beta
        self.rho = rho
        self.nu = nu

    @staticmethod
    def sigma_black_scholes(F, K, T, alpha, beta, rho, nu):
        # simplified to facilitate easy vectorization
        z = (nu / alpha) * ((F * K) ** (0.5 * (1 - beta))) * np.log(F / K)
        zhi = np.log((((1 - 2 * rho * z + z * z) ** 0.5) + z - rho) / (1 - rho))
        numer1 = (((1 - beta) ** 2) / 24) * ((alpha * alpha) / ((F * K) ** (1 - beta)))
        numer2 = 0.25 * rho * beta * nu * alpha / ((F * K) ** ((1 - beta) / 2))
        numer3 = ((2 - 3 * rho * rho) / 24) * nu * nu
        numer = alpha * (1 + (numer1 + numer2 + numer3) * T) * z
        denom1 = ((1 - beta) ** 2 / 24) * (np.log(F / K)) ** 2
        denom2 = (((1 - beta) ** 4) / 1920) * ((np.log(F / K)) ** 4)
        denom = ((F * K) ** ((1 - beta) / 2)) * (1 + denom1 + denom2) * zhi

        return numer / denom

    def cash_nothing_call(self, F=None, K=None, r=None, alpha=None, beta=None, rho=None,
                          nu=None, T=None):
        F, K, r, alpha, beta, rho, nu, T = self._get_params(F=F, K=K, r=r,
                                                            alpha=alpha, beta=beta, rho=rho,  # noqa
                                                            nu=nu, T=T)  # noqa

        sigma_bs = self.sigma_black_scholes(F, K, T, alpha, beta, rho, nu)
        return super().cash_nothing_call(F=F, K=K, r=r, sigma=sigma_bs, T=T) 

    def cash_nothing_put(self, F=None, K=None, r=None, alpha=None, beta=None, rho=None,
                         nu=None, T=None):
        F, K, r, alpha, beta, rho, nu, T = self._get_params(F=F, K=K, r=r,
                                                            alpha=alpha, beta=beta, rho=rho,  # noqa
                                                            nu=nu, T=T)  # noqa

        sigma_bs = self.sigma_black_scholes(F, K, T, alpha, beta, rho, nu)
        return super().cash_nothing_put(F=F, K=K, r=r, sigma=sigma_bs, T=T)

    def asset_nothing_call(self, F=None, K=None, r=None, alpha=None, beta=None, rho=None,
                           nu=None, T=None):
        F, K, r, alpha, beta, rho, nu, T = self._get_params(F=F, K=K, r=r,
                                                            alpha=alpha, beta=beta, rho=rho,  # noqa
                                                            nu=nu, T=T)  # noqa

        sigma_bs = self.sigma_black_scholes(F, K, T, alpha, beta, rho, nu)
        return super().asset_nothing_call(F=F, K=K, r=r, sigma=sigma_bs, T=T)

    def asset_nothing_put(self, F=None, K=None, r=None, alpha=None, beta=None, rho=None,
                          nu=None, T=None):
        F, K, r, alpha, beta, rho, nu, T = self._get_params(F=F, K=K, r=r,
                                                            alpha=alpha, beta=beta, rho=rho,  # noqa
                                                            nu=nu, T=T)  # noqa

        sigma_bs = self.sigma_black_scholes(F, K, T, alpha, beta, rho, nu)
        return super().asset_nothing_put(F=F, K=K, r=r, sigma=sigma_bs, T=T)

In [30]:
alpha = 1.81727308
beta = 0.7
rho = -0.40460926
nu = 2.78934577
sabr_45_days = SABR(F=F, r=r, alpha=alpha, beta=beta, rho=rho, nu=nu, T=T, verbose=False)
sabr_45_days

Black76: SABR
	F: 3663.3762493669747
	K: Not Set
	r: 0.0020510755555555554
	alpha: 1.81727308
	beta: 0.7
	rho: -0.40460926
	nu: 2.78934577
	T: 0.1232876712328767
	

In [31]:
def sabr_put_integrand(K):
    return sabr_45_days.vanilla_put(K=K) * (-2/9) * (K ** (-5/3)) - (1.5/(K**2))

def sabr_call_integrand(K):
    return sabr_45_days.vanilla_call(K=K) * (-2/9) * (K ** (-5/3)) - (1.5/(K**2))

In [32]:
payoff_dc = np.exp(-r*T) * (F ** (1/3)) + 1.5 * np.log(F) + 10
itg_put_sabr, err = sp.integrate.quad(sabr_put_integrand, 0, F)
itg_call_sabr, err = sp.integrate.quad(sabr_call_integrand, F, 6996)

static_valuation = payoff_dc + itg_put_sabr + itg_call_sabr
static_valuation

C:\Users\Miti\AppData\Local\Temp\ipykernel_18576\3687794666.py:2: IntegrationWarning: The integral is probably divergent, or slowly convergent.
  itg_put_sabr, err = sp.integrate.quad(sabr_put_integrand, 0, F)


37.71098680265409

In [33]:
def sabr_put_var_integrand(K):
    return sabr_45_days.vanilla_put(K=K) / (K**2)

def sabr_call_var_integrand(K):
    return sabr_45_days.vanilla_call(K=K) / (K**2)

In [35]:
I_put = quad(sabr_put_var_integrand, 0, F)
I_call = quad(sabr_call_var_integrand, F, 6996)
E_var_SABR = 2*np.exp(r*T)*(I_put[0] + I_call[0])
print('SABR Integrated Variance: %.9f' % E_var_SABR)

SABR Integrated Variance: 0.006350513


In [36]:
sigma_SABR = np.sqrt(E_var_SABR/T)
sigma_SABR # much higher than others!

0.22695751488713725